На основе выбранных продуктов из первого вебинара, выберите 2 A/B-теста и составьте план эксперимента, чтобы избежать Peeking Problem

Распишите, какие метрики и как вы будете анализировать

#### Продукт - ozon.ru

#### Гипотеза:

Увеличение срока хранения товара в точках получения с 7 дней до 12 дней. Позволит снизить показатель Churn rate c 8% до 5% за год.

#### Что делаем:

Контрольная версия: оставляем срок хранения товара без изменений.
Тестовая версия: увеличиваем срок хранения товара в точках получения с 7 дней до 12 дней.

#### На каких пользователях тестируем:

Участвует как действующие, так и новые пользователи платформы. Важно определить, когда клиента можно считать потерянным. За данный показатель возьмем отсутствие покупок в течение календарного месяца с момента последней покупки. Участие принимают точки выдачи заказов в трех городах: Нижний Новгород, Казань, Краснодар. Среднее количество человек, которые совершают покупку в данных городах,  составляет 10000 человек в день.

#### Метрики:

●	Churn rate (Отток клиентов) - primary.

●	Процент товаров, которые не забрали пользователи в период хранения товара — secondary.

#### План действий: 

●	Если наш эксперимент будет положительным и мы зафиксируем ожидаемое уменьшение показателя Churn rate (Отток клиентов) и процент товаров, которые не забрали пользователи в период хранения товара не будет увеличен, то масштабируем изменение и «выкатываем» его на все пункты выдачи заказов по России.

●	Если показатель Churn rate (Отток клиентов) увеличивается или снижается незначительно, либо процент товаров, которые не забрали пользователи увеличивается, то откатываем эксперимент. 


In [16]:
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from typing import Union
import plotly.graph_objects as go
from scipy import stats
from math import asin
import numpy as np

Посчитаем сколько по времени необходимо проводить эксперимент  при alpha = 5% и beta = 20%.
Churn rate (Отток клиентов) - качественная метрика.

In [17]:
cr_1, cr_2 = .08, .05
num_purchases = 10_000

In [18]:
#Расчёт effect_size для пропорций

def calc_proportion_es(prob1: float, prob2: float):
    return abs(proportion_effectsize(prob1, prob2)) 

es_prop = calc_proportion_es(cr_1, cr_2)
es_prop


0.12248629262683436

In [19]:
#Расчёт минимально необходимой выборки * 2(для теста и контроля)

def calc_sample_size_proportion(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)


selection = calc_sample_size_proportion(effect_size=es_prop, alpha=.05, beta=.2)
print(selection)

2092


In [21]:
duration = selection/num_purchases
print(f'duration ~ {duration} day')

duration ~ 0.2092 day


Расчетное значение продолжительности теста (~ 0.2092 дня) не учитывает выполнения целевого действия (получение покупки в пунктах выдачи товаров) с учетом того, что данный показатель был увеличен с 7 дней до 12 дней. Также стоит учесть, что клиент считается "потерянным" при отсутствии покупок в течение календарного месяца с момента последней покупки. Т.о. минимальный возможный срок проведения тестирования составляет календарный месяц с момента запуска. 